***OPENAI LLM setup***

**Loading packages, libraries and secrets into notebook**

In [ ]:
%run ../Setup.ipynb

***Chain setup***

In [4]:
# Model and parsing setup
model = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")
parser = StrOutputParser()

# Define prompt template
template = """
Provide first a natural language Translation followed by an Explanation of the SQL statement. Go through it step by step and use the information of the Context as examples and the schema to get the names of tables and columns. If you can't answer the question, reply "I don't know".

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# Chain setup
chain_3b = (
    {"question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)

# Execute the chain with the logging retriever
chain_3b.invoke("SELECT T1.Name ,  sum(T2.Sales) FROM singer AS T1 JOIN song AS T2 ON T1.Singer_ID  =  T2.Singer_ID GROUP BY T1.Name")

'Translation: Show the name of each singer along with the total sales of their songs. Join the "singer" table (aliased as T1) with the "song" table (aliased as T2) on the condition that the Singer_ID in both tables matches. Then, group the results by the name of the singer.\n\nExplanation:\n1. SELECT T1.Name, sum(T2.Sales): This part of the statement selects the Name column from the T1 table (singer) and calculates the sum of the Sales column from the T2 table (song).\n\n2. FROM singer AS T1 JOIN song AS T2 ON T1.Singer_ID = T2.Singer_ID: This part specifies the tables involved in the query and how they are related. It joins the singer table (aliased as T1) with the song table (aliased as T2) based on the condition that the Singer_ID in both tables is equal.\n\n3. GROUP BY T1.Name: This part groups the results based on the Name column from the T1 table (singer), so that the total sales are calculated for each singer.\n\nOverall, this SQL statement retrieves the names of singers along w

***Chat interface setup***

In [5]:
# Define the chain_invoke function
def chain_3b_invoke(question):
    # Execute the chain with the logging retriever
    result = chain_3b.invoke(question)
    # Return the result 
    return result

# Create a web interface for the app, using Gradio
with gr.Blocks(theme=Base(), title="Question Answering App using Vector Search + RAG") as demo:
    gr.Markdown(
        """
        # Question Answering App using Atlas Vector Search + RAG Architecture
        """)
    textbox = gr.Textbox(label="Enter your SQL statement:")
    with gr.Row():
        button = gr.Button("Submit", variant="primary")
    output = gr.Textbox(lines=1, max_lines=30, label="Natural language translation and explanation:")

# Call chain_invoke function upon clicking the Submit button
    button.click(chain_3b_invoke, textbox, outputs=output)

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
